In [1]:
import pandas as pd

df = pd.read_json("combined.json")

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2314712 entries, 0 to 2314711
Data columns (total 3 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   body            object
 1   subreddit       object
 2   subreddit_full  object
dtypes: object(3)
memory usage: 53.0+ MB


In [3]:
df = df.drop(['subreddit_full'], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2314712 entries, 0 to 2314711
Data columns (total 2 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   body       object
 1   subreddit  object
dtypes: object(2)
memory usage: 35.3+ MB


# Data cleaning

## 1 - Removing duplicates

In [4]:
df.describe()

,body,subreddit
count,2314712,2314712
unique,2152515,192
top,[removed],r-WinStupidPrizes
freq,17732,79296


In [5]:
duplicates_df = df[df.duplicated(subset='body', keep=False)].sort_values(by='body').reset_index(drop=True)
print(f"Number of duplicated rows (including originals): {len(duplicates_df)}")
print("These are the duplicate entries in 'body':")
print(duplicates_df)

# Step 2: Drop duplicates, keeping only the first occurrence
df_1 = df.drop_duplicates(subset='body', keep='first').reset_index(drop=True)
print(f"\nNumber of unique rows after dropping duplicates: {len(df_1)}")
print("Data after removing duplicates:")
print(df_1)

# Step 3: Show counts of how many times each duplicate occurs
duplicate_counts = df['body'].value_counts()
duplicate_counts = duplicate_counts[duplicate_counts > 1]

print("\nCount of each duplicate 'body' text (those that appear more than once):")
for text, count in duplicate_counts.items():
    print(f"'{text[:60]}...' appears {count} times")  # Show first 60 chars to keep output readable
    if count < 100:
        break


Number of duplicated rows (including originals): 190579
These are the duplicate entries in 'body':
       body             subreddit
0                    r-Paranormal
1              r-ShitAmericansSay
2               r-stepdadreflexes
3                     r-thebutton
4               r-WinStupidPrizes
...     ...                   ...
190574   🫶🏼  r-contagiouslaughter
190575   🫶🏼        r-standupshots
190576   🫶🏼           r-dank_meme
190577  🫶🫶🫶      r-wholesomememes
190578  🫶🫶🫶         r-rarepuppers

[190579 rows x 2 columns]

Number of unique rows after dropping duplicates: 2152515
Data after removing duplicates:
                                                      body   subreddit
0                                                    Brick  r-3amjokes
1        Ban wave #2 is complete, please keep reporting...  r-3amjokes
2        I'm not an insomniac, but I am narcoleptic, an...  r-3amjokes
3                      Hear you loud and clear, thank you.  r-3amjokes
4                    

## 2 - Removing links

In [6]:
import re
from html import unescape

In [7]:
def strip_links_html(text, verbose=False):
    if not isinstance(text, str):
        if verbose:
            print(f"Skipping non-string input: {text}")
        return ""

    if verbose:
        print("Original:", text)

    # Unescape HTML entities (like &nbsp;)
    text = unescape(text)
    if verbose:
        print("After unescape:", text)

    # Remove HTML tags
    text = re.sub(r'<[^>]+>', ' ', text)
    if verbose:
        print("After removing HTML tags:", text)

    # Remove URLs
    text = re.sub(r'https?://\S+|www\.\S+', ' ', text)
    if verbose:
        print("After removing URLs:", text)

    # Remove HTML-like codes (&gt;, &amp;)
    text = re.sub(r'&\w+;', ' ', text)
    if verbose:
        print("After removing entities:", text)

    # Normalize spaces
    text = re.sub(r'\s+', ' ', text).strip()
    if verbose:
        print("Final cleaned text:", text)
        print("="*50)

    return text

# Ensure 'df_1' and 'body' column exist
if 'df_1' in locals() and 'body' in df_1.columns:
    df_2 = df_1.copy()
    df_2['body_cleaned'] = df_2['body'].apply(lambda x: strip_links_html(x, verbose=False))
    
    # ✅ Show sample output
    print("Sample cleaned output:\n")
    print(df_2[['body', 'body_cleaned']].head(10))
else:
    print("DataFrame 'df_1' or column 'body' not found.")

Sample cleaned output:

                                                body  \
0                                              Brick   
1  Ban wave #2 is complete, please keep reporting...   
2  I'm not an insomniac, but I am narcoleptic, an...   
3                Hear you loud and clear, thank you.   
4                   I had no idea this sub had mods.   
5  can I be in the subreddit at any other time th...   
6    Have a random question: who is on the sub icon?   
7                               r/notinteresting lol   
8                                        😍😍😍🥰🥰🥰🥰🥰🥰😗😗   
9       B döndüm döne dk d dans d dlcn dllflgmglnfmf   

                                        body_cleaned  
0                                              Brick  
1  Ban wave #2 is complete, please keep reporting...  
2  I'm not an insomniac, but I am narcoleptic, an...  
3                Hear you loud and clear, thank you.  
4                   I had no idea this sub had mods.  
5  can I be in the subreddit 

## 3 - Removing non-useful

In [8]:
# Create a mask to find 'r/...' patterns across all columns
print("🔎 Searching for 'r/...' patterns in all columns...")
mask_r_style = df_2.apply(lambda x: x.astype(str).str.contains(r'\br\/\w+', regex=True)).any(axis=1)
print(f"Found {mask_r_style.sum()} rows containing 'r/...' patterns.")

# Step 2: Remove those rows and store them separately
removed_df = df_2[mask_r_style].copy()
df_3 = df_2[~mask_r_style].copy()
print(f"Rows removed: {len(removed_df)}")
print(f"Rows remaining after cleaning: {len(df_3)}")

# Step 3: Show sample rows
print("\n🔍 Example of removed rows (contain 'r/...'):")
print(removed_df[['body_cleaned']].head(2))

print("\nExample of cleaned rows (without 'r/...'):")
print(df_3[['body_cleaned']].head(2))

🔎 Searching for 'r/...' patterns in all columns...
Found 34070 rows containing 'r/...' patterns.
Rows removed: 34070
Rows remaining after cleaning: 2118445

🔍 Example of removed rows (contain 'r/...'):
                           body_cleaned
7                  r/notinteresting lol
18  Is this r/antijokes but with jokes?

Example of cleaned rows (without 'r/...'):
                                        body_cleaned
0                                              Brick
1  Ban wave #2 is complete, please keep reporting...


In [9]:
df_3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2118445 entries, 0 to 2152514
Data columns (total 3 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   body          object
 1   subreddit     object
 2   body_cleaned  object
dtypes: object(3)
memory usage: 64.6+ MB


In [14]:
# Index reset
data = df_3.copy()
data = df_3.reset_index(drop=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2118445 entries, 0 to 2118444
Data columns (total 3 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   body          object
 1   subreddit     object
 2   body_cleaned  object
dtypes: object(3)
memory usage: 48.5+ MB


# Save data in chunks

### Save csv with 'tilda' as separator

In [19]:
def save_df_as_csv(df, filename):
    with open(filename, "w", encoding="utf-8") as f:
        # Write header row
        f.write("~".join(f'"{col}"' for col in df.columns) + "\n")
        
        # Write each data row
        for _, row in df.iterrows():
            quoted = []
            for item in row:
                # Convert to string and clean
                val = str(item).replace('\n', '.').replace('\r', '.').replace('"', '""')
                quoted.append(f'"{val}"')  # Wrap in double quotes
            f.write("~".join(quoted) + "\n")

# Split and save in 5 chunks
chunk_size = 400000
total_rows = len(data)

for i in range(5):
    start = i * chunk_size
    end = start + chunk_size if i < 4 else total_rows  # last chunk takes the rest
    chunk_df = data.iloc[start:end]
    filename = f"precleaned_chunk_{i+1}.csv"
    save_df_as_csv(chunk_df, filename)
    print(f"Saved: {filename} ({len(chunk_df)} rows)\n\n")
    chunk_df.info()
    print("\n\n")


Saved: precleaned_chunk_1.csv (400000 rows)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400000 entries, 0 to 399999
Data columns (total 3 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   body          400000 non-null  object
 1   subreddit     400000 non-null  object
 2   body_cleaned  400000 non-null  object
dtypes: object(3)
memory usage: 9.2+ MB



Saved: precleaned_chunk_2.csv (400000 rows)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400000 entries, 400000 to 799999
Data columns (total 3 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   body          400000 non-null  object
 1   subreddit     400000 non-null  object
 2   body_cleaned  400000 non-null  object
dtypes: object(3)
memory usage: 9.2+ MB



Saved: precleaned_chunk_3.csv (400000 rows)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400000 entries, 800000 to 1199999
Data columns (total 3 columns):
 #   